In [2]:
import os
from pathlib import Path

cwd = os.getcwd()
root_dir = Path(cwd)
codeformer_dir = root_dir / "CodeFormer"
wav2lip_dir = root_dir / "Wav2Lip"
images_dir = root_dir / "images"
output_dir = root_dir / "output"
input_dir = root_dir / "input"

codeformer_results_dir = codeformer_dir / "results"
wav2lip_result_voice_path = wav2lip_dir / "results" / "result_voice.mp4"

input_audio_path = input_dir / "audio.wav"

output_batch_avi_path = output_dir / "batch_0000.avi"
output_final_mp4_path = output_dir / "final.mp4"

CODEFORMER_FIDELITY = 0.6

print(f"root dir: {root_dir}")
print(f"CodeFormer dir: {codeformer_dir}")
print(f"Wav2Lip dir: {wav2lip_dir}")
print(f"images dir: {images_dir}")
print(f"input dir: {input_dir}")
print(f"output dir: {output_dir}")
print(f"CodeFormer->results dir: {codeformer_results_dir}")
print(f"Wav2Lip->result_voice.mp4 path: {wav2lip_result_voice_path}")
print(f"input->audio path: {input_audio_path}")
print(f"output->batch_0000.avi path: {output_batch_avi_path}")
print(f"output->final.mp4 path: {output_final_mp4_path}")


root dir: /root/repositories/motion-wav2lip-codeformer
CodeFormer dir: /root/repositories/motion-wav2lip-codeformer/CodeFormer
Wav2Lip dir: /root/repositories/motion-wav2lip-codeformer/Wav2Lip
images dir: /root/repositories/motion-wav2lip-codeformer/images
input dir: /root/repositories/motion-wav2lip-codeformer/input
output dir: /root/repositories/motion-wav2lip-codeformer/output
CodeFormer->results dir: /root/repositories/motion-wav2lip-codeformer/CodeFormer/results
Wav2Lip->result_voice.mp4 path: /root/repositories/motion-wav2lip-codeformer/Wav2Lip/results/result_voice.mp4
output->audio path: /root/repositories/motion-wav2lip-codeformer/output/batch_0000.avi
output->batch_0000.avi path: /root/repositories/motion-wav2lip-codeformer/output/batch_0000.avi
output->final.mp4 path: /root/repositories/motion-wav2lip-codeformer/output/final.mp4


In [2]:
%rm -rf $images_dir
%mkdir $images_dir

%rm -rf $codeformer_results_dir
%mkdir $codeformer_results_dir

In [3]:
%cd $wav2lip_dir
!python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face '../input/driving.mp4' --audio '../input/generated.wav'

/root/repositories/motion-wav2lip-codeformer/Wav2Lip
Using cuda for inference.
Reading video frames...
Number of frames available for inference: 141
/root/repositories/motion-wav2lip-codeformer/Wav2Lip/audio.py:100: FutureWarning: Pass sr=16000, n_fft=800 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.filters.mel(hp.sample_rate, hp.n_fft, n_mels=hp.num_mels,
(80, 539)
Length of mel chunks: 198
100%|█████████████████████████████████████████████| 9/9 [00:08<00:00,  1.12it/s]
Load checkpoint from: checkpoints/wav2lip_gan.pth
Model loaded
100%|█████████████████████████████████████████████| 2/2 [00:14<00:00,  7.25s/it]
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-

In [3]:
%cd $wav2lip_dir

import cv2
from tqdm import tqdm
from os import path
import os

vidcap = cv2.VideoCapture(str(wav2lip_result_voice_path))
numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("FPS: ", fps, "Frames: ", numberOfFrames)

for frameNumber in tqdm(range(numberOfFrames)):
    _,image = vidcap.read()
    frame_output_path = str(images_dir / str(str(frameNumber).zfill(4)+'.jpg'))
    cv2.imwrite(frame_output_path, image)

/root/repositories/motion-wav2lip-codeformer/Wav2Lip
FPS:  29.916666666666668 Frames:  198


100%|██████████| 198/198 [00:00<00:00, 1154.90it/s]


In [2]:
%cd $codeformer_dir

import cv2
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

/root/repositories/motion-wav2lip-codeformer/CodeFormer


In [5]:
%cd $codeformer_dir

!python inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path /root/repositories/Wav2Lip-CodeFormer/images --bg_upsampler realesrgan

/root/repositories/motion-wav2lip-codeformer/CodeFormer
Face detection model: retinaface_resnet50
Background upsampling: True, Face upsampling: False
[1/198] Processing: 0000.jpg
	detect 1 faces
[2/198] Processing: 0001.jpg
	detect 1 faces
[3/198] Processing: 0002.jpg
	detect 1 faces
[4/198] Processing: 0003.jpg
	detect 1 faces
[5/198] Processing: 0004.jpg
	detect 1 faces
[6/198] Processing: 0005.jpg
	detect 1 faces
[7/198] Processing: 0006.jpg
	detect 1 faces
[8/198] Processing: 0007.jpg
	detect 1 faces
[9/198] Processing: 0008.jpg
	detect 1 faces
[10/198] Processing: 0009.jpg
	detect 1 faces
[11/198] Processing: 0010.jpg
	detect 1 faces
[12/198] Processing: 0011.jpg
	detect 1 faces
[13/198] Processing: 0012.jpg
	detect 1 faces
[14/198] Processing: 0013.jpg
	detect 1 faces
[15/198] Processing: 0014.jpg
	detect 1 faces
[16/198] Processing: 0015.jpg
	detect 1 faces
[17/198] Processing: 0016.jpg
	detect 1 faces
[18/198] Processing: 0017.jpg
	detect 1 faces
[19/198] Processing: 0018.jpg
	

In [21]:
!rm $output_batch_avi_path
!rm $output_final_mp4_path

In [20]:
import os
import ffmpeg

img_fidelity_dir = "images_"+str(CODEFORMER_FIDELITY)
restored_frames_dir = codeformer_results_dir / str(img_fidelity_dir) / 'final_results'

dir_list = os.listdir(restored_frames_dir)
dir_list.sort()

import cv2
import numpy as np

batch = 0
batchSize = 600
from tqdm import tqdm
for i in tqdm(range(0, len(dir_list), batchSize)):
  img_array = []
  start, end = i, i+batchSize
  print("processing ", start, end)
  for filename in  tqdm(dir_list[start:end]):
      filename = str(restored_frames_dir / str(filename))
      img = cv2.imread(filename)
      if img is None:
        continue
      height, width, layers = img.shape
      size = (width,height)
      img_array.append(img)


  out = cv2.VideoWriter(str(output_dir)+'/batch_'+str(batch).zfill(4)+'.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
  batch = batch + 1
 
  for i in range(len(img_array)):
    out.write(img_array[i])
  out.release()

audio = ffmpeg.input(str(input_audio_path))
video = ffmpeg.input(str(output_batch_avi_path))
out = ffmpeg.output(video, audio, str(output_final_mp4_path))
out.run()  

  0%|          | 0/1 [00:00<?, ?it/s]

processing  0 600


100%|██████████| 1/1 [00:03<00:00,  3.89s/it]
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvid

(None, None)